<div style="text-align: center; background-color: #559cff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
   Final Project - Introduction To Data Science 
</div>

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Data Collection and Preprocessing
</div>

## Import

In [25]:
import pandas as pd
import requests
import json

## Data Collection

Our group chose the website https://www.communitybenefitinsight.org/?page=info.data_api to collect our data. This website contains information about hospitals across the United States. 
The Community Benefit Insight data API allows us to retrieve the following types of data:
+ Hospital data (optionally filtered by state)
+ Detailed data about a single hospital

### Retrieve hospital data

The link https://www.communitybenefitinsight.org/api/get_hospitals.php gives brief data about all hospitals in the United States.

In [26]:
url = 'https://www.communitybenefitinsight.org/api/get_hospitals.php'
data = requests.get(url).text
hospital_data = json.loads(data)
attribute_names = list(hospital_data[0].keys())
data = {}
for attribute in attribute_names:
    data[attribute] = [item[attribute] for item in hospital_data]
df = pd.DataFrame.from_dict(data)
df.to_csv('../data/full_hospital.csv', index=False)

### Retrieve detailed data from every hospital

To retrieve data for each hospital, we use https://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id= + ID (ID is the hospital ID)

Get attributes for the dataset 

In [27]:
url = 'https://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id=1'
data = requests.get(url).text
web = json.loads(data)
attribute = list(web[-1].keys())
attribute

['hospital_data_id',
 'fiscal_yr',
 'data_ein',
 'data_name',
 'form_990_filed_part_grp_ret_f',
 'form_990_num_fac_oper',
 'form_990_name',
 'form_990_address',
 'form_990_city',
 'form_990_state',
 'form_990_zip',
 'tot_func_exp',
 'tot_revenue',
 'tot_comm_bnfts',
 'chrty_care',
 'unreim_medcd',
 'unreim_costs',
 'comm_hlth_impr_svcs_comm_bnft_oper',
 'hlth_prof_educ',
 'subsd_hlth_svcs',
 'rsrch',
 'cash_inknd_contrib_comm_grps',
 'comm_bldg',
 'comm_bldg_actvs',
 'comm_bldg_actvs_physimprvhse',
 'comm_bldg_actvs_econdevlp',
 'comm_bldg_actvs_cmntysuprt',
 'comm_bldg_actvs_envrnimprv',
 'comm_bldg_actvs_ldrdevlp',
 'comm_bldg_actvs_cltnbldg',
 'comm_bldg_actvs_htlhimprvadvcy',
 'comm_bldg_actvs_wrkfrcdevlp',
 'comm_bldg_actvs_other',
 'comm_bldg_actvs_physimprvhse_pct',
 'comm_bldg_actvs_econdevlp_pct',
 'comm_bldg_actvs_cmntysuprt_pct',
 'comm_bldg_actvs_envrnimprv_pct',
 'comm_bldg_actvs_ldrdevlp_pct',
 'comm_bldg_actvs_cltnbldg_pct',
 'comm_bldg_actvs_htlhimprvadvcy_pct',
 'comm_

Create a dictionary to store value for each attribute. The dictionary has one extra key to store the hospital id.

In [28]:
single_data = {}
single_data['hospital_id'] = []
for name in attribute:
    single_data[name] = []

There are total 3491 hospitals in the dataset but the website only allowed us to make 100 requests per week. So I had to change my VPN to retrieve information about 100 hospitals at a time by changing loop range and running the cell repeatedly.

The data about a specific hospital is updated every year so our group only retrieved the data of the latest year.


```
for id in range(1, 100):
    new_url = 'https://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id=' + str(id)
    new_data = requests.get(new_url).text
    json_data = json.loads(new_data)
    for name in attribute:
        single_data[name].append(json_data[-1][name])
    single_data['hospital_id'].append(id)
```

After changing the loop range multiple times, single_data would store the detailed data about all hospitals in it. We save the dictionary as csv format.

```
df = pd.DataFrame.from_dict(single_data)
df.to_csv('../data/single_hospital.csv', index=False)
```

Finally we will concatenate that dataset to the first one based on the hospital_id attribute.

In [29]:
first_df = pd.read_csv('../data/full_hospital.csv')
second_df = pd.read_csv('../data/single_hospital.csv')
raw_df = pd.merge(first_df, second_df, on='hospital_id')
raw_df.to_csv('../data/raw_data.csv', index = False)
raw_df.head(5)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,...,pctttlexp_econdevlp,pctttlexp_cmntysuprt,pctttlexp_envrnimprv,pctttlexp_ldrdevlp,pctttlexp_cltnbldg,pctttlexp_htlhimprvadvcy,pctttlexp_wrkfrcdevlp,pctttlexp_other,pctttlexp_total,updated_dt_y
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,1039,...,0.0,0.0001,0.0,0.0,0.0,0.0001,0.0017,0.0,0.0019,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,1073,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,1117,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,1073,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,1019,...,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,"November 20, 2023"


## Data Preprocessing & Exploration

### Data Preprocessing

In [30]:
raw_df = pd.read_csv('../data/raw_data.csv')

#### Check for duplicate row in data 

In [31]:
index = raw_df.index
detectDupSeries = index.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()

if num_duplicated_rows == 0:
    print(f"Data have no duplicated line.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

Data have no duplicated line.!


#### Choosing attribute

There are 161 features in the dataframe which is too many so our goal is to reduce it down to at least 30-40 features.

First we find the columns that has NaN value.

In [32]:
nan_count = raw_df.isna().sum()

columns_with_NaN_value = nan_count[nan_count > 0]
len(columns_with_NaN_value)

41

There are 41 columns in the dataset that has NaN value. So we will delete columns that 20% of them are NaN values.

In [33]:
chosen_columns = nan_count[nan_count > raw_df.shape[0]*0.2]
del_columns = chosen_columns.index.tolist()
for name in del_columns:
    del raw_df[name]
raw_df.shape

(3491, 144)

We see there are a lot of columns in the dataframe that has a lot of 0 value. So we will choose which one to eliminate.

In [34]:
columns_with_0 = raw_df.columns[(raw_df == 0).sum() > 0]
len(columns_with_0)

101

In [35]:
columns_with_0 = raw_df.columns[(raw_df == 0).sum() > raw_df.shape[0]*0.2]
len(columns_with_0)

76

In [36]:
columns_with_0 = raw_df.columns[(raw_df == 0).sum() > raw_df.shape[0]*0.5]
len(columns_with_0)

64

There are over 101 columns that filled with 0 so we will try our best to remove as many columns as possible. There is not much difference between the number of columns with 50% zero and the number of columns with 20% zero so we will choose threshold 20% so we can remove more columns.

In [37]:
drop_columns = raw_df.columns[(raw_df == 0).sum() > raw_df.shape[0]*0.2]
raw_df = raw_df.drop(columns=drop_columns)
raw_df.shape

(3491, 68)

The dataframe now only has 68 columns. Next our group will eliminate attributes that our group think it is irrelevant.

In [38]:
del raw_df['hospital_org_id'], raw_df['hospital_data_id'], raw_df['updated_dt_x'], raw_df['updated_dt_y']
del raw_df['zip_code'], raw_df['fips_state_and_county_code'], raw_df['data_ein'], raw_df['name_cr'], raw_df['data_name'], raw_df['hospital_bed_size']
del raw_df['form_990_name'], raw_df['form_990_address'], raw_df['form_990_city'], raw_df['form_990_state'], raw_df['form_990_zip']
del raw_df['per_capita_inc_qrt'], raw_df['med_hsehld_inc_qrt'], raw_df['percent_ppl_pov_qrt'], raw_df['percent_ppl_65_yrs_old_without_hlth_insr_qrt'], raw_df['unemp_rate_16_yrs_older_qrt']

- hospital_org_id and hospital_data_id are similar to hospital_id so we delete it.
- updated_dt_x and updated_dt_y are the day the record was updated so it is irrelevant.
- zip_code and fips_state_and_country_code are irrelevant here because we already have hospital_id and name to differentiate each hospital.
- name_cr and data_name are basically similar to name, data_ein is similar to ein.
- The attributes start with 'form_990' that I selected are just similar to the columns we already have.
- We don't really need the attributes that end with '_qrt' because we already have them in number so we don't need to calculate it in quartile.

Many columns are just flag, so we will select carefully attributes to delete

In [39]:
columns_with_Y = raw_df.columns[(raw_df == 'Y').sum() > 0]
for x in columns_with_Y:
    print(x)
    print(raw_df[x].value_counts()['Y'])

chrch_affl_f
347
urban_location_f
2288
children_hospital_f
101
memb_counc_teach_hosps_f
1099
form_990_filed_part_grp_ret_f
1481
sole_comm_prov_f
25
written_fncl_asst_policy_f
3484
used_fed_pov_gdlns_det_elig_free_disc_care_f
3473
chna_cond_comm_hlth_needs_assmt_f
3316
chna_defined_comm_served_f
3314
chna_took_into_acct_input_ppl_rep_broad_intrst_comm_served_f
3313
chna_cond_w_mult_hosp_facilities_f
2004
chna_made_wide_avail_pub_f
3318
hosp_adp_imp_stg_addr_hlth_needs_comm_f
3284
st_impl_aca_medcd_exp_cvrg_adlts_w_incs_up_138pct_fpl_f
2205
st_law_req_hosp_rpt_comm_bnfts_f
2480
paper_return_f
65


If the values in column are divided so unevenly we will delete it because I think that's when the attribute is less effective. If the column is 80% Yes or No we will delete it.

In [40]:
drop_columns_Y = raw_df.columns[(raw_df == 'Y').sum() > raw_df.shape[0]*0.8]
raw_df = raw_df.drop(columns=drop_columns_Y)

drop_columns_N = raw_df.columns[(raw_df == 'N').sum() > raw_df.shape[0]*0.8]
raw_df = raw_df.drop(columns=drop_columns_N)

raw_df.shape

(3491, 37)

Next we need to fill in NaN value in those columns that have it.

In [41]:
nan_count = raw_df.isna().sum()
columns_with_NaN_value = nan_count[nan_count > 0]
columns_with_NaN_value

county                                                   1
tot_comm_bnfts_tot_func_exp_pct                          7
chrty_care_tot_func_exp_pct                              7
unreim_medcd_tot_func_exp_pct                            7
comm_hlth_impr_svcs_comm_bnft_oper_tot_func_exp_pct      7
bad_debt_tot_func_exp_pct                                7
mdcre_shrtfl_tot_func_exp_pct                            7
rat_pat_care_npat_care_comm_bnfts                      266
chna_cond_w_mult_hosp_facilities_f                     169
dtype: int64

There are some columns that have type object so we will exclude them when we fill in the NaN value with the median of the column.

In [42]:
nan_count = raw_df.isna().sum()
columns_with_NaN_value = nan_count[nan_count > 0]
columns_with_NaN_value = columns_with_NaN_value.index.to_list()
num_df = raw_df.select_dtypes(exclude='object')
fill_col = num_df.columns.intersection(columns_with_NaN_value)
for name in fill_col:
    raw_df[name] = raw_df[name].fillna(raw_df[name].median())

We save the data that has been preprocessed as final_data.csv.

In [43]:
raw_df.to_csv('../data/final_data.csv', index=False)

### Data exploration

In [44]:
df = pd.read_csv('../data/final_data.csv')

#### How many rows and how many columns does the raw data have?

In [45]:
shape = df.shape
print(f"The data has {shape[0]} rows and {shape[1]} columns.")

The data has 3491 rows and 37 columns.


#### What data type does each column currently have?

In [46]:
df.dtypes

hospital_id                                                  int64
ein                                                          int64
name                                                        object
street_address                                              object
city                                                        object
state                                                       object
hospital_bed_count                                           int64
urban_location_f                                            object
memb_counc_teach_hosps_f                                    object
medicare_provider_number                                     int64
county                                                      object
fiscal_yr                                                    int64
form_990_filed_part_grp_ret_f                               object
form_990_num_fac_oper                                        int64
tot_func_exp                                                 i

The data type looks suitable for us to explore.

#### What does each column mean?

In [47]:
col_meaning_df = pd.read_csv('../data/data_schema.csv')
# col_meaning_df.set_index('Column', inplace=True)
pd.set_option('display.max_colwidth', 350)

col_meaning_df

,Column,Meaning
0,hospital_id,Internal hospital ID
1,ein,Hospital EIN
2,name,Hospital name
3,street_address,Street address
4,city,City
5,state,State
6,hospital_bed_count,Number of beds
7,urban_location_f,Urban location flag
8,memb_counc_teach_hosps_f,Member of the Council of Teaching Hospitals flag
9,county,Hospital county


#### For each column with numeric data type, how are the values distributed?

For columns with numeric data types, we calculated:
- Percentage (from 0 to 100) of missing values
- The min
- The lower quartile (phân vị 25)
- The median (phân vị 50)
- The upper quartile (phân vị 75)
- The max

In [48]:
num_col_info_df = df.select_dtypes(exclude='object')
del num_col_info_df['hospital_id']
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,ein,hospital_bed_count,medicare_provider_number,fiscal_yr,form_990_num_fac_oper,tot_func_exp,tot_revenue,tot_comm_bnfts,chrty_care,unreim_medcd,...,unreim_medcd_tot_func_exp_pct,comm_hlth_impr_svcs_comm_bnft_oper_tot_func_exp_pct,bad_debt_tot_func_exp_pct,mdcre_shrtfl_tot_func_exp_pct,rat_pat_care_npat_care_comm_bnfts,per_capita_inc,med_hsehld_inc,percent_ppl_pov_pct,percent_ppl_65_yrs_old_without_hlth_insr_pct,unemp_rate_16_yrs_older
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
min,10130427.0,2.0,10007.0,2010.0,1.0,0.000000e+00,-1.615266e+08,-1.877057e+06,-7511212.0,-4.359135e+07,...,-100000.000000,-0.018872,-1.174136,-81.982677,-3372.579913,0.0,0.0,0.0,0.0,0.0
lower_quartile,352528741.0,32.0,141333.0,2021.0,1.0,6.311005e+07,6.592734e+07,4.182806e+06,464754.0,7.136190e+05,...,0.900000,0.000000,0.900000,-4.300000,0.900000,44490.0,51720.0,9.4,7.0,4.0
median,476028103.0,114.0,261335.0,2021.0,1.0,2.940063e+08,3.093186e+08,2.235120e+07,3236355.0,7.412280e+06,...,3.300000,0.100000,2.300000,-1.700000,2.500000,51141.0,60070.0,12.4,9.5,5.0
upper_quartile,741356589.0,275.0,390074.5,2021.0,5.0,1.292430e+09,1.423750e+09,1.267807e+08,18159097.0,4.264596e+07,...,6.000000,0.300000,4.700000,0.100000,6.800000,60138.0,71799.0,15.4,13.2,6.3
max,990269825.0,3060.0,670309.0,2021.0,43.0,2.881186e+10,3.222840e+10,1.411503e+09,333132504.0,1.089481e+09,...,742.167399,42593.461997,100000.000000,100000.000000,22391.329545,191220.0,155362.0,44.7,35.8,19.8


#### For each column with categorical data type, how are the values distributed?

In [49]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

cat_col_info_df = df.select_dtypes(include='object')

def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def num_values(s):
    s = s.str.split(';')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.str.split(';')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,name,street_address,city,state,urban_location_f,memb_counc_teach_hosps_f,county,form_990_filed_part_grp_ret_f,chna_cond_w_mult_hosp_facilities_f,st_impl_aca_medcd_exp_cvrg_adlts_w_incs_up_138pct_fpl_f,st_law_req_hosp_rpt_comm_bnfts_f
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.8,0.0,0.0
num_values,3330,3439,2192,51,2,2,1195,2,2,2,2
value_ratios,"{'Mercy Medical Center': 0.2, 'St Marys Hospital': 0.2, 'St Francis Hospital': 0.2, 'St Josephs ...","{'100 Hospital Drive': 0.1, 'One Hospital Drive': 0.1, '100 Medical Center Drive': 0.1, '200 Hos...","{'Chicago': 0.7, 'Philadelphia': 0.5, 'Baltimore': 0.4, 'Columbus': 0.4, 'New York': 0.4, 'Dalla...","{'CA': 6.9, 'TX': 6.5, 'NY': 5.3, 'PA': 5.2, 'OH': 4.6, 'IL': 4.4, 'MI': 3.8, 'WI': 3.8, 'MN': 3...","{'Y': 65.5, 'N': 34.5}","{'N': 68.5, 'Y': 31.5}","{'Los Angeles County': 1.5, 'Cook County': 1.3, 'Jefferson County': 1.1, 'Montgomery County': 1....","{'N': 57.6, 'Y': 42.4}","{'Y': 60.3, 'N': 39.7}","{'Y': 63.2, 'N': 36.8}","{'Y': 71.0, 'N': 29.0}"
